# Google Sheets Inventory Control (Python + Notebook)

This notebook shows how to READ and WRITE Google Sheets data for an inventory use case.

## Key Point About *API Keys*
An API Key alone CANNOT modify (write to) private Google Sheets. API Keys only allow limited, usually **read-only access to public** Sheets endpoints.

To WRITE (update / append) you must use either:
1. **Service Account credentials** (best for automation / CLI).
2. OAuth Installed App credentials (per end-user auth).

Below we implement the **Service Account** path (recommended). We still show (later) a minimal *read-only with API key* example for public sheets.

---
### What You'll Achieve
- Configure credentials via environment variables or a `.env` file.
- List current inventory rows.
- Append a stock movement / new item.
- Update a specific cell/range.
- Provide a small CLI script (`update_sheet.py`) you can call from the command line.

### Placeholders You Must Replace
- `YOUR_SHEET_ID_HERE` with the actual Spreadsheet ID (the long hash in the URL).
- Ensure the target worksheet/tab names match your sheet (default examples use `Items`).

---
### Folder / File Expectations
You will add a Service Account JSON key file (NOT committed) and point an environment variable at it.
`service_account.json` (example name) should be kept outside version control (add to `.gitignore`).

---

## 1. Enable APIs & Create Credentials
1. Go to https://console.cloud.google.com/ 
2. Create / select a project.
3. Enable: **Google Sheets API** and (optionally for Drive file discovery) **Google Drive API**.
4. Create a **Service Account** (IAM & Admin > Service Accounts).
5. Create a key (JSON) for the Service Account and download it (e.g. `service_account.json`).
6. Share your target Google Sheet with the **service account email** (e.g. `my-sa@project.iam.gserviceaccount.com`) granting at least Editor access.

Environment variable to set (CLI):
- `GOOGLE_APPLICATION_CREDENTIALS` = absolute path to your JSON key file.
- `GSHEET_INVENTORY_ID` = your spreadsheet ID.

Optional: Create a `.env` file with these lines: 
```

In [1]:
# 2. Install required libraries (run once)
%pip install --quiet gspread google-auth google-auth-httplib2 google-api-python-client python-dotenv tabulate

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 3. Load environment variables (.env support)
import os, json, datetime, time
from pathlib import Path
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

CREDENTIALS_PATH = os.getenv('GOOGLE_APPLICATION_CREDENTIALS', 'gudetama-bible-studio_service-account.json')
SPREADSHEET_ID = os.getenv('GSHEET_INVENTORY_ID', 'YOUR_SHEET_ID_HERE')

print('Credentials file exists:', Path(CREDENTIALS_PATH).exists())
print('Spreadsheet ID set:', SPREADSHEET_ID != 'YOUR_SHEET_ID_HERE')

Credentials file exists: True
Spreadsheet ID set: True


In [2]:
# 4. Initialize clients (gspread + raw Sheets API)
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
import gspread

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
if not Path(CREDENTIALS_PATH).exists():
    raise FileNotFoundError(f'Missing credentials file: {CREDENTIALS_PATH}')

creds = Credentials.from_service_account_file(CREDENTIALS_PATH, scopes=SCOPES)
sheets_service = build('sheets', 'v4', credentials=creds)
gc = gspread.authorize(creds)

# Access spreadsheet (will error if ID invalid or not shared)
sh = gc.open_by_key(SPREADSHEET_ID)
print('Opened spreadsheet title:', sh.title)

Opened spreadsheet title: Test plug in của Sophia


## 5. Helper functions
Assumes an `Items` worksheet with header row like: `ID | Name | SKU | Qty | ReorderLevel | UpdatedAt`. Adjust indices as needed.

In [3]:
from typing import List, Dict, Any, Optional

ITEMS_SHEET_NAME = 'Items'  # change if different
HEADER_ROW_INDEX = 1

def get_items_ws():
    return sh.worksheet(ITEMS_SHEET_NAME)

def list_items() -> List[Dict[str, Any]]:
    ws = get_items_ws()
    rows = ws.get_all_values()
    if not rows:
        return []
    headers = rows[HEADER_ROW_INDEX-1]
    items = []
    for r in rows[HEADER_ROW_INDEX:]:
        if all(c.strip()=='' for c in r):
            continue
        item = {h: (r[i] if i < len(r) else '') for i, h in enumerate(headers)}
        items.append(item)
    return items

def print_items_table():
    from tabulate import tabulate
    items = list_items()
    if not items:
        print('No items found.')
        return
    headers = items[0].keys()
    print(tabulate([[i[h] for h in headers] for i in items], headers=headers, tablefmt='github'))

def find_item_row_by_sku(sku: str) -> Optional[int]:
    ws = get_items_ws()
    col_values = ws.col_values(3)  # assuming column 3 is SKU
    for idx, val in enumerate(col_values, start=1):
        if val.strip().lower() == sku.strip().lower():
            return idx
    return None

def update_quantity_by_sku(sku: str, delta: int):
    row = find_item_row_by_sku(sku)
    if not row:
        raise ValueError(f'SKU {sku} not found')
    ws = get_items_ws()
    qty_cell = ws.cell(row, 4)  # column 4 is Qty
    try:
        current = int(qty_cell.value) if qty_cell.value else 0
    except ValueError:
        raise ValueError(f'Non-integer quantity in row {row}: {qty_cell.value}')
    new_qty = current + delta
    ws.update_cell(row, 4, new_qty)
    ws.update_cell(row, 6, datetime.datetime.utcnow().isoformat(timespec='seconds')+'Z')  # UpdatedAt col 6
    return new_qty

def append_item(item: Dict[str, Any]):
    ws = get_items_ws()
    headers = ws.row_values(HEADER_ROW_INDEX)
    row = [item.get(h, '') for h in headers]
    ws.append_row(row, value_input_option='USER_ENTERED')
    return row

print('Helper functions loaded.')

Helper functions loaded.


In [4]:
# 6. Demo: list items (will print table if any).
print_items_table()

| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             |                      |
| 20000             |                      |


In [ ]:
# 7. Demo: adjust quantity for a SKU (uncomment & set an existing SKU)
# new_qty = update_quantity_by_sku('SKU123', +5)
# print('Updated quantity =', new_qty)

In [ ]:
# Add this cell after the existing helper functions

def analyze_sheet_structure(detailed: bool = False) -> Dict[str, Any]:
    """
    Analyze the complete structure of the Google Sheet
    
    Args:
        detailed: If True, includes sample data from each worksheet
    
    Returns:
        Dictionary containing structure information
    """
    structure = {
        'spreadsheet_title': sh.title,
        'spreadsheet_id': SPREADSHEET_ID,
        'worksheets': [],
        'total_worksheets': 0
    }
    
    try:
        # Get all worksheets
        worksheets = sh.worksheets()
        structure['total_worksheets'] = len(worksheets)
        
        for ws in worksheets:
            ws_info = {
                'name': ws.title,
                'id': ws.id,
                'row_count': ws.row_count,
                'col_count': ws.col_count,
                'headers': [],
                'sample_data': [],
                'empty_rows': 0,
                'data_rows': 0
            }
            
            try:
                # Get all values
                all_values = ws.get_all_values()
                
                if all_values:
                    # Headers (first row)
                    headers = all_values[0] if all_values else []
                    ws_info['headers'] = [h.strip() for h in headers]
                    
                    # Count data rows and empty rows
                    data_rows = []
                    empty_count = 0
                    
                    for i, row in enumerate(all_values[1:], start=2):  # Skip header
                        if any(cell.strip() for cell in row):
                            data_rows.append(row)
                            ws_info['data_rows'] += 1
                        else:
                            empty_count += 1
                    
                    ws_info['empty_rows'] = empty_count
                    
                    # Sample data (first 3 data rows if detailed)
                    if detailed and data_rows:
                        ws_info['sample_data'] = data_rows[:3]
                
            except Exception as e:
                ws_info['error'] = f"Could not read worksheet data: {str(e)}"
            
            structure['worksheets'].append(ws_info)
            
    except Exception as e:
        structure['error'] = f"Could not access spreadsheet: {str(e)}"
    
    return structure

def print_sheet_structure(detailed: bool = False):
    """Print a formatted analysis of the sheet structure"""
    from tabulate import tabulate
    
    print("=" * 80)
    print("📊 GOOGLE SHEETS STRUCTURE ANALYSIS")
    print("=" * 80)
    
    structure = analyze_sheet_structure(detailed)
    
    if 'error' in structure:
        print(f"❌ Error: {structure['error']}")
        return
    
    print(f"📋 Spreadsheet: {structure['spreadsheet_title']}")
    print(f"🆔 ID: {structure['spreadsheet_id']}")
    print(f"📄 Total Worksheets: {structure['total_worksheets']}")
    print()
    
    for i, ws in enumerate(structure['worksheets'], 1):
        print(f"📑 Worksheet {i}: {ws['name']}")
        print("-" * 60)
        
        if 'error' in ws:
            print(f"   ❌ Error: {ws['error']}")
            continue
        
        print(f"   📏 Dimensions: {ws['row_count']} rows × {ws['col_count']} columns")
        print(f"   📊 Data rows: {ws['data_rows']}")
        print(f"   🔍 Empty rows: {ws['empty_rows']}")
        
        if ws['headers']:
            print(f"   📋 Headers ({len(ws['headers'])}):")
            for idx, header in enumerate(ws['headers'], 1):
                print(f"      {idx:2d}. {header or '(empty)'}")
        else:
            print("   📋 No headers found")
        
        if detailed and ws['sample_data']:
            print(f"   📄 Sample Data:")
            headers = ws['headers'] or [f"Col{i}" for i in range(1, len(ws['sample_data'][0])+1)]
            print(tabulate(ws['sample_data'], headers=headers, tablefmt='grid', maxcolwidths=15))
        
        print()

def check_column_types(worksheet_name: str = None) -> Dict[str, Dict[str, Any]]:
    """
    Analyze data types in each column of a worksheet
    
    Args:
        worksheet_name: Name of worksheet to analyze. If None, uses ITEMS_SHEET_NAME
    
    Returns:
        Dictionary with column analysis
    """
    ws_name = worksheet_name or ITEMS_SHEET_NAME
    
    try:
        ws = sh.worksheet(ws_name)
        all_values = ws.get_all_values()
        
        if not all_values:
            return {'error': 'No data found'}
        
        headers = all_values[0]
        data_rows = all_values[1:]
        
        column_analysis = {}
        
        for col_idx, header in enumerate(headers):
            col_data = [row[col_idx] if col_idx < len(row) else '' for row in data_rows]
            col_data = [cell.strip() for cell in col_data if cell.strip()]  # Remove empty
            
            analysis = {
                'header': header.strip(),
                'total_values': len(col_data),
                'empty_cells': len(data_rows) - len(col_data),
                'unique_values': len(set(col_data)),
                'data_types': {},
                'sample_values': col_data[:5] if col_data else [],
                'potential_issues': []
            }
            
            # Analyze data types
            type_counts = {'integer': 0, 'float': 0, 'date': 0, 'text': 0}
            
            for value in col_data:
                if value.isdigit() or (value.startswith('-') and value[1:].isdigit()):
                    type_counts['integer'] += 1
                elif value.replace('.', '').replace('-', '').isdigit():
                    type_counts['float'] += 1
                elif any(sep in value for sep in ['/', '-', ':']):
                    # Simple date detection
                    type_counts['date'] += 1
                else:
                    type_counts['text'] += 1
            
            analysis['data_types'] = type_counts
            
            # Identify potential issues
            if analysis['empty_cells'] > len(data_rows) * 0.5:
                analysis['potential_issues'].append('High number of empty cells')
            
            if header.strip().lower() in ['qty', 'quantity', 'stock'] and type_counts['text'] > 0:
                analysis['potential_issues'].append('Non-numeric values in quantity column')
            
            if 'id' in header.strip().lower() and analysis['unique_values'] != analysis['total_values']:
                analysis['potential_issues'].append('Duplicate values in ID column')
            
            column_analysis[f'col_{col_idx+1}_{header.strip()}'] = analysis
            
    except Exception as e:
        return {'error': f'Could not analyze worksheet: {str(e)}'}
    
    return column_analysis

def print_column_analysis(worksheet_name: str = None):
    """Print formatted column analysis"""
    analysis = check_column_types(worksheet_name)
    
    if 'error' in analysis:
        print(f"❌ Error: {analysis['error']}")
        return
    
    ws_name = worksheet_name or ITEMS_SHEET_NAME
    print(f"📊 COLUMN ANALYSIS: {ws_name}")
    print("=" * 70)
    
    for col_key, col_info in analysis.items():
        print(f"📋 {col_info['header']} (Column {col_key.split('_')[1]})")
        print(f"   📊 Total values: {col_info['total_values']}")
        print(f"   🔍 Empty cells: {col_info['empty_cells']}")
        print(f"   🎯 Unique values: {col_info['unique_values']}")
        
        # Data types
        types = col_info['data_types']
        type_summary = [f"{k}: {v}" for k, v in types.items() if v > 0]
        print(f"   📝 Data types: {', '.join(type_summary)}")
        
        # Sample values
        if col_info['sample_values']:
            samples = ', '.join([f'"{v}"' for v in col_info['sample_values']])
            print(f"   📄 Samples: {samples}")
        
        # Issues
        if col_info['potential_issues']:
            for issue in col_info['potential_issues']:
                print(f"   ⚠️  Issue: {issue}")
        
        print()
        



print('📊 Sheet structure analysis functions loaded.')

📊 Sheet structure analysis functions loaded.


In [11]:
# Usage examples - add these as separate cells

# Basic structure analysis
print_sheet_structure()

📊 GOOGLE SHEETS STRUCTURE ANALYSIS
📋 Spreadsheet: Test plug in của Sophia
🆔 ID: 1oDsfVNyen-0TQZR03ZU5H_d1232EpMUuTVkOdxKlKmo
📄 Total Worksheets: 6

📑 Worksheet 1: Suppliers
------------------------------------------------------------
   📏 Dimensions: 5 rows × 8 columns
   📊 Data rows: 3
   🔍 Empty rows: 0
   📋 Headers (8):
       1. idSheetFournisseurs
       2. (empty)
       3. (empty)
       4. (empty)
       5. (empty)
       6. (empty)
       7. (empty)
       8. (empty)

📑 Worksheet 2: Items
------------------------------------------------------------
   📏 Dimensions: 5 rows × 10 columns
   📊 Data rows: 4
   🔍 Empty rows: 0
   📋 Headers (10):
       1. idSheetArticles
       2. (empty)
       3. (empty)
       4. (empty)
       5. (empty)
       6. (empty)
       7. (empty)
       8. (empty)
       9. (empty)
      10. (empty)

📑 Worksheet 3: Stock
------------------------------------------------------------
   📏 Dimensions: 5 rows × 7 columns
   📊 Data rows: 2
   🔍 Empty rows: 0

In [12]:
# Detailed structure with sample data
print_sheet_structure(detailed=True)

📊 GOOGLE SHEETS STRUCTURE ANALYSIS
📋 Spreadsheet: Test plug in của Sophia
🆔 ID: 1oDsfVNyen-0TQZR03ZU5H_d1232EpMUuTVkOdxKlKmo
📄 Total Worksheets: 6

📑 Worksheet 1: Suppliers
------------------------------------------------------------
   📏 Dimensions: 5 rows × 8 columns
   📊 Data rows: 3
   🔍 Empty rows: 0
   📋 Headers (8):
       1. idSheetFournisseurs
       2. (empty)
       3. (empty)
       4. (empty)
       5. (empty)
       6. (empty)
       7. (empty)
       8. (empty)
   📄 Sample Data:
+-----------------------+-----------------+-----------------+-----------------+--------------+------------+-----------------+-----------+
| idSheetFournisseurs   |                 |                 |                 |              |            |                 |           |
+=======================+=================+=================+=================+==============+============+=================+===========+
| idfournisseur+t       | nomfournisseur+ | adressemail+tex | telephone+numbe | adresse

In [13]:
# Column type analysis for Items sheet
print_column_analysis()

📊 COLUMN ANALYSIS: Items
📋 idSheetArticles (Column 1)
   📊 Total values: 4
   🔍 Empty cells: 0
   🎯 Unique values: 4
   📝 Data types: integer: 2, text: 2
   📄 Samples: "idarticle+text", "Item ID", "10000", "20000"

📋  (Column 2)
   📊 Total values: 4
   🔍 Empty cells: 0
   🎯 Unique values: 4
   📝 Data types: text: 4
   📄 Samples: "nomarticle+text", "Item Name", "Pencil", "Eraser"

📋  (Column 3)
   📊 Total values: 4
   🔍 Empty cells: 0
   🎯 Unique values: 4
   📝 Data types: text: 4
   📄 Samples: "description+text", "Description", "Bút chì màu đen dùng để cho học sinh ghi bài", "Tẩy bút chì một đầu"

📋  (Column 4)
   📊 Total values: 4
   🔍 Empty cells: 0
   🎯 Unique values: 3
   📝 Data types: text: 4
   📄 Samples: "categorie+text", "Category", "Dụng cụ học tập", "Dụng cụ học tập"

📋  (Column 5)
   📊 Total values: 4
   🔍 Empty cells: 0
   🎯 Unique values: 3
   📝 Data types: integer: 2, text: 2
   📄 Samples: "prixunitaireht+number", "Unit Price", "5000", "5000"

📋  (Column 6)
   📊 Total val

📋 INVENTORY SCHEMA VALIDATION
✅ Valid schema: False
✅ Found columns: id
❌ Missing columns: name, sku, qty
⚠️  Issues:
   - Recommended column missing: reorderlevel
   - Recommended column missing: safetystock
   - Recommended column missing: updatedat


In [19]:
# Vietnamese Inventory Item Management Functions
# Based on your actual column structure analysis

def add_vietnamese_item(
    item_id: str,
    item_name: str, 
    description: str,
    category: str,
    unit_price: int,
    currency: str = "VND",
    storage_unit: int = 1,
    barcode: str = "",
    supplier_id: str = "",
    safety_stock: int = 0
) -> Dict[str, Any]:
    """
    Add a new item to the Vietnamese inventory sheet
    
    Based on your column structure:
    Col1: idSheetArticles (Item ID)
    Col2: (Item Name) 
    Col3: (Description)
    Col4: (Category)
    Col5: (Unit Price)
    Col6: (Currency)
    Col7: (Storage Unit)
    Col8: (Barcode)
    Col9: (Supplier ID) 
    Col10: (Safety Stock)
    """
    
    ws = get_items_ws()
    
    # Check if item ID already exists
    existing_ids = ws.col_values(1)[1:]  # Skip header
    if item_id in existing_ids:
        raise ValueError(f'Item ID {item_id} already exists')
    
    # Create row data in exact column order (1-10)
    row_data = [
        item_id,           # Column 1: Item ID
        item_name,         # Column 2: Item Name
        description,       # Column 3: Description
        category,          # Column 4: Category
        str(unit_price),   # Column 5: Unit Price
        currency,          # Column 6: Currency
        str(storage_unit), # Column 7: Storage Unit
        barcode,           # Column 8: Barcode
        supplier_id,       # Column 9: Supplier ID
        str(safety_stock) if safety_stock else ''  # Column 10: Safety Stock
    ]
    
    # Append the row
    ws.append_row(row_data, value_input_option='USER_ENTERED')
    
    print(f'✅ Added item: {item_name} (ID: {item_id})')
    return {
        'item_id': item_id,
        'item_name': item_name,
        'description': description,
        'category': category,
        'unit_price': unit_price,
        'currency': currency,
        'barcode': barcode,
        'supplier': supplier_id
    }

def add_sample_vietnamese_items():
    """Add sample Vietnamese school supply items"""
    
    sample_items = [
        {
            'item_id': '30000',
            'item_name': 'Sổ tay học sinh',
            'description': 'Sổ tay 200 trang có kẻ ngang dành cho học sinh tiểu học',
            'category': 'Dụng cụ học tập',
            'unit_price': 15000,
            'barcode': '123456781',
            'supplier_id': 'Thiên Long',
            'safety_stock': 50
        },
        {
            'item_id': '40000', 
            'item_name': 'Bút bi màu xanh',
            'description': 'Bút bi nước màu xanh, mực không phai',
            'category': 'Dụng cụ học tập',
            'unit_price': 8000,
            'barcode': '123456782',
            'supplier_id': 'Thiên Long', 
            'safety_stock': 100
        },
        {
            'item_id': '50000',
            'item_name': 'Thước kẻ 30cm',
            'description': 'Thước kẻ nhựa trong suốt 30cm có chia độ chính xác',
            'category': 'Dụng cụ học tập',
            'unit_price': 12000,
            'barcode': '123456783', 
            'supplier_id': 'Thiên Long',
            'safety_stock': 30
        },
        {
            'item_id': '60000',
            'item_name': 'Keo dán UHU',
            'description': 'Keo dán đa năng 40ml không độc hại cho trẻ em',
            'category': 'Dụng cụ học tập',
            'unit_price': 25000,
            'barcode': '123456784',
            'supplier_id': 'UHU Vietnam',
            'safety_stock': 20
        },
        {
            'item_id': '70000',
            'item_name': 'Bộ màu chì 12 màu',
            'description': 'Bộ chì màu 12 cây nhiều màu sắc cho học sinh',
            'category': 'Dụng cụ học tập', 
            'unit_price': 45000,
            'barcode': '123456785',
            'supplier_id': 'Thiên Long',
            'safety_stock': 25
        }
    ]
    
    added_items = []
    for item in sample_items:
        try:
            result = add_vietnamese_item(**item)
            added_items.append(result)
        except ValueError as e:
            print(f'⚠️  Skipped {item["item_name"]}: {str(e)}')
        except Exception as e:
            print(f'❌ Error adding {item["item_name"]}: {str(e)}')
    
    return added_items

def add_custom_vietnamese_item():
    """Interactive function to add a custom item via input prompts"""
    
    print("📝 Adding new Vietnamese inventory item")
    print("-" * 40)
    
    try:
        item_id = input("Item ID (e.g. 80000): ").strip()
        if not item_id:
            raise ValueError("Item ID cannot be empty")
            
        item_name = input("Item Name (Vietnamese): ").strip()
        if not item_name:
            raise ValueError("Item name cannot be empty")
            
        description = input("Description (Vietnamese): ").strip()
        category = input("Category (default: Dụng cụ học tập): ").strip() or "Dụng cụ học tập"
        
        unit_price_str = input("Unit Price (VND): ").strip()
        try:
            unit_price = int(unit_price_str)
        except ValueError:
            raise ValueError("Unit price must be a number")
            
        currency = input("Currency (default: VND): ").strip() or "VND"
        
        storage_unit_str = input("Storage Unit (default: 1): ").strip() or "1"
        try:
            storage_unit = int(storage_unit_str)
        except ValueError:
            storage_unit = 1
            
        barcode = input("Barcode (optional): ").strip()
        supplier_id = input("Supplier ID (default: Thiên Long): ").strip() or "Thiên Long"
        
        safety_stock_str = input("Safety Stock (default: 0): ").strip() or "0"
        try:
            safety_stock = int(safety_stock_str)
        except ValueError:
            safety_stock = 0
        
        # Add the item
        result = add_vietnamese_item(
            item_id=item_id,
            item_name=item_name,
            description=description, 
            category=category,
            unit_price=unit_price,
            currency=currency,
            storage_unit=storage_unit,
            barcode=barcode,
            supplier_id=supplier_id,
            safety_stock=safety_stock
        )
        
        print(f"\n✅ Successfully added: {item_name}")
        return result
        
    except KeyboardInterrupt:
        print("\n❌ Cancelled by user")
        return None
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        return None

def batch_add_items_from_list(items_list: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Add multiple items from a list of dictionaries
    
    Args:
        items_list: List of item dictionaries with required fields
    """
    added_items = []
    failed_items = []
    
    print(f"📦 Batch adding {len(items_list)} items...")
    
    for i, item in enumerate(items_list, 1):
        try:
            print(f"  {i}/{len(items_list)}: {item.get('item_name', 'Unknown')}")
            result = add_vietnamese_item(**item)
            added_items.append(result)
        except Exception as e:
            error_info = {
                'item': item,
                'error': str(e)
            }
            failed_items.append(error_info)
            print(f"  ❌ Failed: {str(e)}")
    
    print(f"\n📊 Batch Results:")
    print(f"   ✅ Successfully added: {len(added_items)}")
    print(f"   ❌ Failed: {len(failed_items)}")
    
    if failed_items:
        print(f"\n🔍 Failed Items:")
        for fail in failed_items:
            print(f"   - {fail['item'].get('item_name', 'Unknown')}: {fail['error']}")
    
    return {
        'added': added_items,
        'failed': failed_items
    }

print('🇻🇳 Vietnamese inventory item management functions loaded.')

🇻🇳 Vietnamese inventory item management functions loaded.


In [21]:
# Example Usage: Add Vietnamese inventory items

# 1. Add a single custom item
result = add_vietnamese_item(
    item_id='80001',
    item_name='Bảng nhỏ học sinh', 
    description='Bảng nhỏ từ tính có bút viết bảng dành cho học sinh',
    category='Dụng cụ học tập',
    unit_price=35000,
    barcode='123456786',
    supplier_id='Thiên Long',
    safety_stock=15
)
print('Added:', result)

print("📋 Available functions:")
print("1. add_vietnamese_item() - Add single item")
print("2. add_sample_vietnamese_items() - Add 5 sample items")  
print("3. add_custom_vietnamese_item() - Interactive item addition")
print("4. batch_add_items_from_list() - Batch add from list")
print("\n💡 Uncomment examples below to test")

✅ Added item: Bảng nhỏ học sinh (ID: 80001)
Added: {'item_id': '80001', 'item_name': 'Bảng nhỏ học sinh', 'description': 'Bảng nhỏ từ tính có bút viết bảng dành cho học sinh', 'category': 'Dụng cụ học tập', 'unit_price': 35000, 'currency': 'VND', 'barcode': '123456786', 'supplier': 'Thiên Long'}
📋 Available functions:
1. add_vietnamese_item() - Add single item
2. add_sample_vietnamese_items() - Add 5 sample items
3. add_custom_vietnamese_item() - Interactive item addition
4. batch_add_items_from_list() - Batch add from list

💡 Uncomment examples below to test


In [22]:
# Add sample Vietnamese items (uncomment to run)
added_items = add_sample_vietnamese_items()
print(f"\n📊 Added {len(added_items)} sample items")

# View the updated inventory
print("📋 Current inventory after adding items:")
print_items_table()

✅ Added item: Sổ tay học sinh (ID: 30000)
✅ Added item: Bút bi màu xanh (ID: 40000)
✅ Added item: Bút bi màu xanh (ID: 40000)
✅ Added item: Thước kẻ 30cm (ID: 50000)
✅ Added item: Thước kẻ 30cm (ID: 50000)
✅ Added item: Keo dán UHU (ID: 60000)
✅ Added item: Keo dán UHU (ID: 60000)
✅ Added item: Bộ màu chì 12 màu (ID: 70000)

📊 Added 5 sample items
📋 Current inventory after adding items:
✅ Added item: Bộ màu chì 12 màu (ID: 70000)

📊 Added 5 sample items
📋 Current inventory after adding items:
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 200                  |
| 80001             | 15                   |
| 30000             | 50                   |
| 40000             | 100                  |
| 50000             | 30                   |
| 60000             | 20                   |
| 70000

In [24]:
# Add this cell after the Vietnamese inventory functions

def update_vietnamese_item_by_id(
    item_id: str,
    **updates
) -> Dict[str, Any]:
    """
    Update an existing item in the Vietnamese inventory sheet by Item ID
    
    Args:
        item_id: The Item ID to find and update
        **updates: Dictionary of field updates (item_name, description, category, etc.)
    
    Available update fields:
    - item_name: Item Name (Column 2)
    - description: Description (Column 3) 
    - category: Category (Column 4)
    - unit_price: Unit Price (Column 5)
    - currency: Currency (Column 6)
    - storage_unit: Storage Unit (Column 7)
    - barcode: Barcode (Column 8)
    - supplier_id: Supplier ID (Column 9)
    - safety_stock: Safety Stock (Column 10)
    """
    
    ws = get_items_ws()
    
    # Find the row with matching Item ID
    id_column_values = ws.col_values(1)  # Column 1 is Item ID
    target_row = None
    
    for idx, value in enumerate(id_column_values, start=1):
        if value.strip() == str(item_id).strip():
            target_row = idx
            break
    
    if not target_row:
        raise ValueError(f'Item ID {item_id} not found in inventory')
    
    if target_row == 1:  # Header row
        raise ValueError(f'Cannot update header row')
    
    # Column mapping for updates
    column_mapping = {
        'item_name': 2,
        'description': 3,
        'category': 4,
        'unit_price': 5,
        'currency': 6,
        'storage_unit': 7,
        'barcode': 8,
        'supplier_id': 9,
        'safety_stock': 10
    }
    
    updated_fields = []
    
    # Apply updates
    for field_name, new_value in updates.items():
        if field_name not in column_mapping:
            print(f'⚠️  Warning: Unknown field "{field_name}" ignored')
            continue
        
        column_num = column_mapping[field_name]
        
        # Convert to string for Google Sheets
        if field_name in ['unit_price', 'storage_unit', 'safety_stock']:
            cell_value = str(new_value) if new_value is not None else ''
        else:
            cell_value = str(new_value) if new_value else ''
        
        # Update the cell
        ws.update_cell(target_row, column_num, cell_value)
        updated_fields.append(f'{field_name}: "{new_value}"')
    
    if updated_fields:
        print(f'✅ Updated Item ID {item_id}:')
        for field in updated_fields:
            print(f'   - {field}')
    else:
        print(f'⚠️  No valid updates provided for Item ID {item_id}')
    
    # Return current row data
    current_row = ws.row_values(target_row)
    headers = ws.row_values(1)
    
    return {
        'item_id': item_id,
        'row_number': target_row,
        'updated_fields': len(updated_fields),
        'current_data': {h: (current_row[i] if i < len(current_row) else '') 
                        for i, h in enumerate(headers)}
    }

def update_multiple_items(updates_list: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Update multiple items in batch
    
    Args:
        updates_list: List of dictionaries, each containing 'item_id' and update fields
        
    Example:
        updates = [
            {'item_id': '10000', 'storage_unit': 12, 'unit_price': 15000},
            {'item_id': '20000', 'storage_unit': 6, 'supplier_id': 'New Supplier'}
        ]
    """
    
    updated_items = []
    failed_items = []
    
    print(f"📦 Batch updating {len(updates_list)} items...")
    
    for i, update_data in enumerate(updates_list, 1):
        item_id = update_data.get('item_id')
        if not item_id:
            failed_items.append({'error': 'Missing item_id', 'data': update_data})
            print(f"  {i}/{len(updates_list)}: ❌ Missing item_id")
            continue
        
        # Extract item_id and get the rest as updates
        updates = {k: v for k, v in update_data.items() if k != 'item_id'}
        
        try:
            print(f"  {i}/{len(updates_list)}: Updating Item ID {item_id}")
            result = update_vietnamese_item_by_id(item_id, **updates)
            updated_items.append(result)
        except Exception as e:
            error_info = {
                'item_id': item_id,
                'error': str(e),
                'updates': updates
            }
            failed_items.append(error_info)
            print(f"  ❌ Failed: {str(e)}")
    
    print(f"\n📊 Batch Update Results:")
    print(f"   ✅ Successfully updated: {len(updated_items)}")
    print(f"   ❌ Failed: {len(failed_items)}")
    
    if failed_items:
        print(f"\n🔍 Failed Updates:")
        for fail in failed_items:
            print(f"   - Item ID {fail.get('item_id', 'Unknown')}: {fail['error']}")
    
    return {
        'updated': updated_items,
        'failed': failed_items
    }

def get_item_by_id(item_id: str) -> Optional[Dict[str, Any]]:
    """Get a specific item by its ID"""
    
    ws = get_items_ws()
    id_column_values = ws.col_values(1)
    
    for idx, value in enumerate(id_column_values, start=1):
        if value.strip() == str(item_id).strip():
            if idx == 1:  # Header row
                continue
            row_data = ws.row_values(idx)
            headers = ws.row_values(1)
            return {
                'row_number': idx,
                'data': {h: (row_data[i] if i < len(row_data) else '') 
                        for i, h in enumerate(headers)}
            }
    
    return None

print('🔄 Vietnamese inventory update functions loaded.')

🔄 Vietnamese inventory update functions loaded.


In [25]:
# Example Usage: Update storage units for first 2 items

# First, let's see what items we have
print("📋 Current inventory:")
print_items_table()
print("\n" + "="*60)

# Method 1: Update individual items
print("🔄 Method 1: Individual Updates")
print("-" * 30)

# Update storage unit for first item (assuming Item ID '10000' exists)
try:
    result1 = update_vietnamese_item_by_id(
        item_id='10000',
        storage_unit=12,  # 12 pieces per box
        unit_price=18000  # Also update price while we're at it
    )
    print(f"Updated: {result1['updated_fields']} fields")
except Exception as e:
    print(f"❌ Error updating Item ID 10000: {str(e)}")

print()

# Update storage unit for second item (assuming Item ID '20000' exists)  
try:
    result2 = update_vietnamese_item_by_id(
        item_id='20000', 
        storage_unit=6,   # 6 pieces per pack
        supplier_id='Thiên Long',
        safety_stock=30
    )
    print(f"Updated: {result2['updated_fields']} fields")
except Exception as e:
    print(f"❌ Error updating Item ID 20000: {str(e)}")

📋 Current inventory:
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 200                  |
| 80001             | 15                   |
| 30000             | 50                   |
| 40000             | 100                  |
| 50000             | 30                   |
| 60000             | 20                   |
| 70000             | 25                   |

🔄 Method 1: Individual Updates
------------------------------
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 200                  |
| 80001             | 15                   |
| 30000             | 50                   |
| 40000         

In [26]:
# Method 2: Batch update multiple items at once
print("\n🔄 Method 2: Batch Updates")
print("-" * 30)

# Define updates for multiple items
storage_updates = [
    {
        'item_id': '10000',
        'storage_unit': 24,  # Change to 24 per carton
        'category': 'Dụng cụ văn phòng'  # Update category too
    },
    {
        'item_id': '20000', 
        'storage_unit': 10,  # Change to 10 per bundle
        'unit_price': 25000,  # Update price
        'barcode': 'UPD123456'  # Update barcode
    }
]

# Execute batch update
batch_result = update_multiple_items(storage_updates)

print(f"\n📊 Batch update summary:")
print(f"   Items updated: {len(batch_result['updated'])}")
print(f"   Items failed: {len(batch_result['failed'])}")


🔄 Method 2: Batch Updates
------------------------------
📦 Batch updating 2 items...
  1/2: Updating Item ID 10000
✅ Updated Item ID 10000:
   - storage_unit: "24"
   - category: "Dụng cụ văn phòng"
  2/2: Updating Item ID 20000
✅ Updated Item ID 20000:
   - storage_unit: "10"
   - unit_price: "25000"
   - barcode: "UPD123456"

📊 Batch Update Results:
   ✅ Successfully updated: 2
   ❌ Failed: 0

📊 Batch update summary:
   Items updated: 2
   Items failed: 0


In [27]:
# Method 3: Verify updates and show final table
print("\n🔍 Method 3: Verify Updates")
print("-" * 30)

# Check specific items
item1 = get_item_by_id('10000')
if item1:
    print(f"📋 Item 10000 details:")
    for key, value in item1['data'].items():
        if value.strip():  # Only show non-empty fields
            print(f"   {key}: {value}")
else:
    print("❌ Item 10000 not found")

print()

item2 = get_item_by_id('20000') 
if item2:
    print(f"📋 Item 20000 details:")
    for key, value in item2['data'].items():
        if value.strip():  # Only show non-empty fields
            print(f"   {key}: {value}")
else:
    print("❌ Item 20000 not found")

print("\n📋 Final Updated Inventory:")
print_items_table()


🔍 Method 3: Verify Updates
------------------------------
📋 Item 10000 details:
   idSheetArticles: 10000

📋 Item 20000 details:
   idSheetArticles: 20000

📋 Final Updated Inventory:
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 30                   |
| 80001             | 15                   |
| 30000             | 50                   |
| 40000             | 100                  |
| 50000             | 30                   |
| 60000             | 20                   |
| 70000             | 25                   |


In [30]:
# Vietnamese Inventory Item Deletion Functions

def delete_vietnamese_item_by_id(
    item_id: str,
    confirm: bool = False
) -> Dict[str, Any]:
    """
    Delete an existing item from the Vietnamese inventory sheet by Item ID
    
    Args:
        item_id: The Item ID to find and delete
        confirm: Set to True to bypass confirmation prompt
    
    Returns:
        Dictionary with deletion result and deleted item data
    """
    
    ws = get_items_ws()
    
    # Find the row with matching Item ID
    id_column_values = ws.col_values(1)  # Column 1 is Item ID
    target_row = None
    
    for idx, value in enumerate(id_column_values, start=1):
        if value.strip() == str(item_id).strip():
            target_row = idx
            break
    
    if not target_row:
        raise ValueError(f'Item ID {item_id} not found in inventory')
    
    if target_row == 1:  # Header row
        raise ValueError(f'Cannot delete header row')
    
    # Get item data before deletion
    current_row = ws.row_values(target_row)
    headers = ws.row_values(1)
    item_data = {h: (current_row[i] if i < len(current_row) else '') 
                 for i, h in enumerate(headers)}
    
    # Safety confirmation (if not bypassed)
    if not confirm:
        print(f"⚠️  WARNING: About to delete item:")
        print(f"   ID: {item_id}")
        print(f"   Name: {item_data.get(headers[1], 'Unknown') if len(headers) > 1 else 'Unknown'}")
        print(f"   Description: {item_data.get(headers[2], 'Unknown') if len(headers) > 2 else 'Unknown'}")
        
        confirmation = input("\n❓ Are you sure you want to delete this item? (yes/no): ").strip().lower()
        if confirmation not in ['yes', 'y']:
            print("❌ Deletion cancelled by user")
            return {
                'deleted': False,
                'reason': 'Cancelled by user',
                'item_data': item_data
            }
    
    # Delete the row
    ws.delete_rows(target_row)
    
    print(f'✅ Deleted Item ID {item_id}: {item_data.get(headers[1], "Unknown") if len(headers) > 1 else "Unknown"}')
    
    return {
        'deleted': True,
        'item_id': item_id,
        'row_number': target_row,
        'item_data': item_data
    }

def delete_multiple_items(
    item_ids: List[str],
    confirm_each: bool = True,
    confirm_batch: bool = True
) -> Dict[str, Any]:
    """
    Delete multiple items by their Item IDs
    
    Args:
        item_ids: List of Item IDs to delete
        confirm_each: Ask confirmation for each item individually
        confirm_batch: Ask confirmation for the entire batch operation
        
    Returns:
        Dictionary with results of batch deletion
    """
    
    if not item_ids:
        return {'deleted': [], 'failed': [], 'cancelled': True}
    
    # Batch confirmation
    if confirm_batch:
        print(f"⚠️  WARNING: About to delete {len(item_ids)} items:")
        for item_id in item_ids:
            print(f"   - Item ID: {item_id}")
        
        confirmation = input(f"\n❓ Proceed with batch deletion of {len(item_ids)} items? (yes/no): ").strip().lower()
        if confirmation not in ['yes', 'y']:
            print("❌ Batch deletion cancelled by user")
            return {'deleted': [], 'failed': [], 'cancelled': True}
    
    deleted_items = []
    failed_items = []
    
    print(f"🗑️  Batch deleting {len(item_ids)} items...")
    
    # Sort item IDs by row number (descending) to avoid shifting row issues
    ws = get_items_ws()
    id_column_values = ws.col_values(1)
    
    # Create list of (row_number, item_id) tuples
    items_with_rows = []
    for item_id in item_ids:
        for idx, value in enumerate(id_column_values, start=1):
            if value.strip() == str(item_id).strip() and idx > 1:  # Skip header
                items_with_rows.append((idx, item_id))
                break
    
    # Sort by row number descending (delete from bottom up)
    items_with_rows.sort(key=lambda x: x[0], reverse=True)
    
    for i, (row_num, item_id) in enumerate(items_with_rows, 1):
        try:
            print(f"  {i}/{len(items_with_rows)}: Deleting Item ID {item_id}")
            
            # Use confirm=True if batch confirmed, or use individual confirmation
            individual_confirm = not confirm_batch if confirm_batch else not confirm_each
            
            result = delete_vietnamese_item_by_id(item_id, confirm=individual_confirm)
            
            if result['deleted']:
                deleted_items.append(result)
            else:
                failed_items.append({
                    'item_id': item_id,
                    'error': result.get('reason', 'Unknown error')
                })
        except Exception as e:
            error_info = {
                'item_id': item_id,
                'error': str(e)
            }
            failed_items.append(error_info)
            print(f"  ❌ Failed: {str(e)}")
    
    # Handle items not found
    found_ids = {item[1] for item in items_with_rows}
    not_found = [item_id for item_id in item_ids if item_id not in found_ids]
    for item_id in not_found:
        failed_items.append({
            'item_id': item_id,
            'error': 'Item ID not found'
        })
    
    print(f"\n📊 Batch Deletion Results:")
    print(f"   ✅ Successfully deleted: {len(deleted_items)}")
    print(f"   ❌ Failed: {len(failed_items)}")
    
    if failed_items:
        print(f"\n🔍 Failed Deletions:")
        for fail in failed_items:
            print(f"   - Item ID {fail['item_id']}: {fail['error']}")
    
    return {
        'deleted': deleted_items,
        'failed': failed_items,
        'cancelled': False
    }

def delete_items_by_criteria(
    criteria: Dict[str, Any],
    confirm: bool = True,
    dry_run: bool = True
) -> Dict[str, Any]:
    """
    Delete items matching specific criteria (advanced deletion)
    
    Args:
        criteria: Dictionary of field:value pairs to match
        confirm: Ask for confirmation before deletion
        dry_run: If True, only show what would be deleted without actually deleting
        
    Example criteria:
        {'category': 'Dụng cụ học tập', 'supplier_id': 'Old Supplier'}
        {'unit_price': '0'}  # Delete items with 0 price
        {'safety_stock': ''}  # Delete items with empty safety stock
    """
    
    ws = get_items_ws()
    all_values = ws.get_all_values()
    
    if not all_values or len(all_values) < 2:
        return {'deleted': [], 'failed': [], 'matches': 0}
    
    headers = all_values[0]
    data_rows = all_values[1:]
    
    # Find matching items
    matching_items = []
    
    for row_idx, row in enumerate(data_rows, start=2):  # Start from row 2 (skip header)
        item_matches = True
        item_data = {h: (row[i] if i < len(row) else '') for i, h in enumerate(headers)}
        
        # Check if row matches all criteria
        for field, value in criteria.items():
            # Find column index for field
            field_col = None
            for col_idx, header in enumerate(headers):
                if header.strip().lower() == field.lower() or field.lower() in header.lower():
                    field_col = col_idx
                    break
            
            if field_col is None:
                print(f"⚠️  Warning: Field '{field}' not found in headers")
                item_matches = False
                break
            
            item_value = row[field_col] if field_col < len(row) else ''
            
            # String comparison (case-insensitive)
            if str(item_value).strip().lower() != str(value).strip().lower():
                item_matches = False
                break
        
        if item_matches:
            matching_items.append({
                'row_number': row_idx,
                'item_id': row[0] if len(row) > 0 else '',
                'item_name': row[1] if len(row) > 1 else '',
                'data': item_data
            })
    
    print(f"🔍 Found {len(matching_items)} items matching criteria:")
    for criterion, value in criteria.items():
        print(f"   {criterion}: '{value}'")
    
    if not matching_items:
        print("   No items found matching the criteria")
        return {'deleted': [], 'failed': [], 'matches': 0}
    
    print(f"\n📋 Matching items:")
    for item in matching_items:
        print(f"   - ID: {item['item_id']}, Name: {item['item_name']}")
    
    if dry_run:
        print(f"\n🔍 DRY RUN: {len(matching_items)} items would be deleted")
        print("   Set dry_run=False to actually delete these items")
        return {'deleted': [], 'failed': [], 'matches': len(matching_items), 'dry_run': True}
    
    if confirm:
        confirmation = input(f"\n❓ Delete {len(matching_items)} matching items? (yes/no): ").strip().lower()
        if confirmation not in ['yes', 'y']:
            print("❌ Deletion cancelled by user")
            return {'deleted': [], 'failed': [], 'matches': len(matching_items), 'cancelled': True}
    
    # Delete matching items (from bottom to top to avoid row shifting)
    matching_items.sort(key=lambda x: x['row_number'], reverse=True)
    item_ids = [item['item_id'] for item in matching_items]
    
    return delete_multiple_items(item_ids, confirm_each=False, confirm_batch=False)

def safe_delete_with_backup(item_id: str) -> Dict[str, Any]:
    """
    Delete an item with automatic backup to a 'Deleted Items' worksheet
    """
    
    # Get the item data first
    item = get_item_by_id(item_id)
    if not item:
        raise ValueError(f'Item ID {item_id} not found')
    
    # Create or get 'Deleted Items' worksheet
    try:
        deleted_ws = sh.worksheet('Deleted Items')
    except:
        # Create the worksheet if it doesn't exist
        deleted_ws = sh.add_worksheet('Deleted Items', rows=1000, cols=12)
        
        # Add headers with timestamp
        headers = get_items_ws().row_values(1)
        headers.append('Deleted At')
        headers.append('Deleted By')
        deleted_ws.append_row(headers)
    
    # Add item to deleted items sheet
    item_row = []
    headers = get_items_ws().row_values(1)
    for header in headers:
        item_row.append(item['data'].get(header, ''))
    
    # Add deletion metadata
    item_row.append(datetime.datetime.utcnow().isoformat(timespec='seconds')+'Z')
    item_row.append('System')  # Could be enhanced with user info
    
    deleted_ws.append_row(item_row)
    
    # Now delete from main sheet
    result = delete_vietnamese_item_by_id(item_id, confirm=True)
    result['backed_up'] = True
    result['backup_sheet'] = 'Deleted Items'
    
    return result

print('🗑️ Vietnamese inventory deletion functions loaded.')

🗑️ Vietnamese inventory deletion functions loaded.


In [31]:
# Example Usage: Delete Vietnamese inventory items

print("🗑️ Available deletion functions:")
print("1. delete_vietnamese_item_by_id() - Delete single item with confirmation")
print("2. delete_multiple_items() - Delete multiple items by ID list") 
print("3. delete_items_by_criteria() - Delete items matching criteria")
print("4. safe_delete_with_backup() - Delete with backup to 'Deleted Items' sheet")
print("\n💡 Examples below (uncomment to test)")

# Example 1: Delete a single item (with confirmation)
print("\n📋 Current inventory before deletion:")
print_items_table()

# Uncomment to delete a specific item
result = delete_vietnamese_item_by_id('80001', confirm=False)  # Skip confirmation
print(f"Deletion result: {result}")

print("\n" + "="*60)

🗑️ Available deletion functions:
1. delete_vietnamese_item_by_id() - Delete single item with confirmation
2. delete_multiple_items() - Delete multiple items by ID list
3. delete_items_by_criteria() - Delete items matching criteria
4. safe_delete_with_backup() - Delete with backup to 'Deleted Items' sheet

💡 Examples below (uncomment to test)

📋 Current inventory before deletion:
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 30                   |
| 80001             | 15                   |
| 30000             | 50                   |
| 40000             | 100                  |
| 50000             | 30                   |
| 60000             | 20                   |
| 70000             | 25                   |
| idSheetArticles   |                      |
|-------------------|------------

In [32]:
# Example 2: Batch delete multiple items
print("🗑️ Example 2: Batch Deletion")
print("-" * 30)

# Define list of Item IDs to delete
items_to_delete = ['30000', '40000', '50000']  # Replace with actual IDs

# Uncomment to execute batch deletion
batch_result = delete_multiple_items(
    item_ids=items_to_delete,
    confirm_each=False,    # Don't ask for each item individually
    confirm_batch=True     # Ask once for the entire batch
)
# 
# print(f"Batch deletion summary:")
print(f"  Deleted: {len(batch_result['deleted'])}")  
print(f"  Failed: {len(batch_result['failed'])}")

print("💡 Uncomment above code to test batch deletion")

🗑️ Example 2: Batch Deletion
------------------------------
⚠️  WARNING: About to delete 3 items:
   - Item ID: 30000
   - Item ID: 40000
   - Item ID: 50000
🗑️  Batch deleting 3 items...
🗑️  Batch deleting 3 items...
  1/3: Deleting Item ID 50000
  1/3: Deleting Item ID 50000
⚠️  WARNING: About to delete item:
   ID: 50000
   Name: Unknown
   Description: Unknown
⚠️  WARNING: About to delete item:
   ID: 50000
   Name: Unknown
   Description: Unknown
✅ Deleted Item ID 50000: Unknown
  2/3: Deleting Item ID 40000
✅ Deleted Item ID 50000: Unknown
  2/3: Deleting Item ID 40000
⚠️  WARNING: About to delete item:
   ID: 40000
   Name: Unknown
   Description: Unknown
⚠️  WARNING: About to delete item:
   ID: 40000
   Name: Unknown
   Description: Unknown
✅ Deleted Item ID 40000: Unknown
  3/3: Deleting Item ID 30000
✅ Deleted Item ID 40000: Unknown
  3/3: Deleting Item ID 30000
⚠️  WARNING: About to delete item:
   ID: 30000
   Name: Unknown
   Description: Unknown
⚠️  WARNING: About to del

In [33]:
# Example 3: Delete items by criteria (advanced)
print("🗑️ Example 3: Criteria-based Deletion")
print("-" * 40)

# Example 3a: Dry run - see what would be deleted
print("🔍 Dry run: Find items with specific supplier")
criteria_result = delete_items_by_criteria(
    criteria={'supplier_id': 'UHU Vietnam'},
    dry_run=True  # Only show matches, don't delete
)
print(f"Found {criteria_result.get('matches', 0)} items matching criteria")

print()

# Example 3b: Delete items with empty safety stock (dry run first)
print("🔍 Dry run: Find items with empty safety stock")
empty_safety_result = delete_items_by_criteria(
    criteria={'safety_stock': ''},  # Empty safety stock
    dry_run=True
)
print(f"Found {empty_safety_result.get('matches', 0)} items with empty safety stock")

print()

# Example 3c: Delete items with 0 price (commented out)
print("🔍 Delete items with 0 price:")
zero_price_result = delete_items_by_criteria(
    criteria={'unit_price': '0'},
    dry_run=False,  # Actually delete
    confirm=True
)

print("💡 Uncomment above code to test criteria deletion")

🗑️ Example 3: Criteria-based Deletion
----------------------------------------
🔍 Dry run: Find items with specific supplier
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
🔍 Found 0 items matching criteria:
   supplier_id: 'UHU Vietnam'
   No items found matching the criteria
Found 0 items matching criteria

🔍 Dry run: Find items with empty safety stock
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
⚠️  Warning: Field 'supplier_id' not found in headers
🔍 Found 0 items matching criteria:
   suppli

In [34]:
# Example 4: Safe deletion with backup
print("🗑️ Example 4: Safe Deletion with Backup")
print("-" * 40)

# This creates a 'Deleted Items' worksheet and backs up the item before deletion
# Uncomment to test safe deletion:

safe_result = safe_delete_with_backup('70000')  # Replace with actual ID
if safe_result['deleted']:
    print(f"✅ Item safely deleted and backed up to '{safe_result['backup_sheet']}' worksheet")
else:
    print(f"❌ Safe deletion failed: {safe_result.get('reason', 'Unknown error')}")

print("💡 Safe deletion creates backup in 'Deleted Items' worksheet")

print("\n" + "="*60)

🗑️ Example 4: Safe Deletion with Backup
----------------------------------------


/tmp/ipykernel_1789/4137453360.py:291: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  item_row.append(datetime.datetime.utcnow().isoformat(timespec='seconds')+'Z')


✅ Deleted Item ID 70000: Unknown
✅ Item safely deleted and backed up to 'Deleted Items' worksheet
💡 Safe deletion creates backup in 'Deleted Items' worksheet



In [35]:
# Example 5: Verify deletions and show final inventory
print("🔍 Example 5: Verification")
print("-" * 25)

# Show current inventory after any deletions
print("📋 Current inventory after deletion operations:")
print_items_table()

print(f"\n📊 Deletion Function Summary:")
print("✅ delete_vietnamese_item_by_id() - Single item deletion with safety prompts")
print("✅ delete_multiple_items() - Batch deletion with confirmation options")  
print("✅ delete_items_by_criteria() - Advanced deletion by field matching")
print("✅ safe_delete_with_backup() - Deletion with automatic backup")

print(f"\n🛡️  Safety Features:")
print("- Confirmation prompts (can be bypassed)")
print("- Row number validation (prevents header deletion)")
print("- Item existence checks")
print("- Dry run mode for criteria-based deletion")
print("- Automatic backup option")
print("- Batch operation error handling")

print(f"\n💡 Pro Tips:")
print("- Always test with dry_run=True first")
print("- Use batch operations for multiple deletions (more efficient)")
print("- Consider safe_delete_with_backup() for important data")
print("- Delete from bottom to top to avoid row shifting issues")

🔍 Example 5: Verification
-------------------------
📋 Current inventory after deletion operations:
| idSheetArticles   |                      |
|-------------------|----------------------|
| idarticle+text    | stocksecurite+number |
| Item ID           | Safety Stock         |
| 10000             | 100                  |
| 20000             | 30                   |
| 60000             | 20                   |

📊 Deletion Function Summary:
✅ delete_vietnamese_item_by_id() - Single item deletion with safety prompts
✅ delete_multiple_items() - Batch deletion with confirmation options
✅ delete_items_by_criteria() - Advanced deletion by field matching
✅ safe_delete_with_backup() - Deletion with automatic backup

🛡️  Safety Features:
- Confirmation prompts (can be bypassed)
- Row number validation (prevents header deletion)
- Item existence checks
- Dry run mode for criteria-based deletion
- Automatic backup option
- Batch operation error handling

💡 Pro Tips:
- Always test with dry_run=Tru

## 9. Optional: Read-Only Access with API Key (Public Sheet)
If your sheet is **publicly published**, you can fetch CSV (no writes):
```
Or use the `values.get` endpoint with an API key:
```
Writes will fail with only an API key. Use service account (above) for modifications.

## 10. Next Steps
- Add validation (no negative stock).
- Create a stock movements sheet & log each delta.
- Batch updates for performance (use `batch_update`).
- Wrap logic into a proper Python package + tests.

Let me know if you want these scaffolded.